# Optimize Initial Conditions
## Yabox

In [1]:
from scipy.optimize import brute
import numpy as np
from datetime import datetime,timedelta
import pandas as pd
from yabox import DE

# Initialize Ray

In [2]:
from environs import Env
env = Env()
env.str("CUDA_DEVICE_ORDER",'PCI_BUS_ID')
env.int("CUDA_VISIBLE_DEVICES",512)
env.int("NUMBA_ENABLE_CUDASIM",1)
env.bool("OMPI_MCA_opal_cuda_support",True)

import os
import ray
MB=1024*1024
GB=MB*1024
ray.shutdown()
ray.init(object_store_memory=1*GB,memory=1*GB,lru_evict=True,\
         driver_object_store_memory=500*MB,num_gpus=500,num_cpus=2, ignore_reinit_error=True) # , include_webui=False, ignore_reinit_error=True)

# ray.init(lru_evict=True,object_store_memory=2*GB,memory=4*GB,
#          num_gpus=312,num_cpus=10, include_webui=False, ignore_reinit_error=True)

# ray.shutdown()
# ray.init(num_cpus=10,num_gpus=None)

@ray.remote(num_gpus=1)
def use_gpu():
    print("ray.get_gpu_ids(): {}".format(ray.get_gpu_ids()[0]))
    print("CUDA_VISIBLE_DEVICES: {}".format(os.environ["CUDA_VISIBLE_DEVICES"]))

use_gpu.remote()

2020-07-06 21:34:50,457	INFO resource_spec.py:212 -- Starting Ray with 0.98 GiB memory available for workers and up to 1.0 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-07-06 21:34:50,919	INFO services.py:1170 -- View the Ray dashboard at localhost:8267


ObjectID(45b95b1c8bd3a9c4ffffffff010000c801000000)

# Load New and Process Data from website data.brasil.io

In [3]:
import get_data
LoadData=True

if LoadData:
    get_data.get_data()

Baixando arquivos brasil.io...
(pid=102642) ray.get_gpu_ids(): 499
(pid=102642) CUDA_VISIBLE_DEVICES: 499


# Functions to Load Processed Data

In [4]:
def load_confirmed(state, start_date):
    dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d')
    df = pd.read_csv('./data/confirmados.csv',delimiter=',',parse_dates=True, date_parser=dateparse)
    y=[]
    x=[]
    for i in range(0,len(df.date)):
        y.append(df[state].values[i])
        x.append(df.date.values[i])
    df2=pd.DataFrame(data=y,index=x,columns=[""])
    df2=df2[start_date:]
    del x,y,df,dateparse
    return df2

def load_dead(state, start_date):
    dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d')
    df = pd.read_csv('./data/mortes.csv',delimiter=',',parse_dates=True, date_parser=dateparse)
    y=[]
    x=[]
    for i in range(0,len(df.date)):
        y.append(df[state].values[i])
        x.append(df.date.values[i])
    df2=pd.DataFrame(data=y,index=x,columns=[""])
    df2=df2[start_date:]
    del x,y,df,dateparse
    return df2

# Load solver

In [5]:
LoadBH=True
if LoadBH:
    import LearnerICRayNoLoadBH as L 
else:
    import LearnerICRayNoLoad as L 

# Data for States

In [6]:
dfparam = pd.read_csv("data/param_optimized_Yabox_HistMin.csv")
dfparam

,state,start-date,prediction-range,s0,e0,a0,i0,r0,d0,START,RATIO,WCASES,WREC
0,SP,2020-03-15,200,9.133723e+06,0,0,713,300,250,100,0.15,0.438537,0.004821
1,ES,2020-04-01,200,5.533807e+05,0,0,55,250,50,100,0.15,0.446217,0.019239
2,MG,2020-04-01,200,1.223239e+06,0,0,223,250,40,100,0.15,0.301378,0.009798
3,RJ,2020-03-20,200,7.976688e+05,0,0,830,250,50,100,0.15,0.666680,0.031825
4,CE,2020-03-20,200,6.746419e+05,0,0,739,250,50,100,0.15,0.552314,0.022969
5,PE,2020-03-20,200,8.251135e+05,0,0,667,250,100,100,0.15,0.435306,0.018729
6,AM,2020-03-20,200,7.931951e+05,0,0,737,250,100,100,0.15,0.582615,0.003338
7,PA,2020-03-20,200,7.807152e+05,0,0,902,250,100,100,0.15,0.530999,0.034237
8,PI,2020-03-20,200,8.251397e+05,0,0,940,250,100,100,0.15,0.318747,0.028869
9,RR,2020-03-20,200,8.086198e+05,0,0,769,250,100,100,0.15,0.392673,0.039875


In [7]:
df = pd.read_csv("data/dados_total_estados.csv")
df

,index,date,state,city,place_type,confirmed,deaths,order_for_place,is_last,popEst,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
0,0,2020-07-06,AP,TOTAL,state,30004,449,108,True,845731.0,16.0,3547.70016,0.0150
1,1,2020-07-05,AP,TOTAL,state,29883,442,107,False,845731.0,16.0,3533.39301,0.0148
2,2,2020-07-04,AP,TOTAL,state,29809,441,106,False,845731.0,16.0,3524.64318,0.0148
3,3,2020-07-03,AP,TOTAL,state,29574,438,105,False,845731.0,16.0,3496.85657,0.0148
4,4,2020-07-02,AP,TOTAL,state,29153,427,104,False,845731.0,16.0,3447.07714,0.0146
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3057,272683,2020-02-29,SP,TOTAL,state,2,0,5,False,45919049.0,35.0,0.00436,0.0000
3058,272684,2020-02-28,SP,TOTAL,state,2,0,4,False,45919049.0,35.0,0.00436,0.0000
3059,272685,2020-02-27,SP,TOTAL,state,1,0,3,False,45919049.0,35.0,0.00218,0.0000
3060,272686,2020-02-26,SP,TOTAL,state,1,0,2,False,45919049.0,35.0,0.00218,0.0000


# Functions for Optimization

In [8]:
from scipy.integrate import odeint
import sys
import io
import gc

def create_f(state,e0,a0,r0,d0,date,startNCases, ratio, predict_range, version):
                
    def fobjective(point):
        
        dead=  load_dead(state,date)
        data = load_confirmed(state,date)*(1-ratio)-dead
        
        cleanRecovered=False
        s0, deltaDate, i0, weigthCases, weigthRecov = point
        end_date=datetime.strptime(date, "%Y-%m-%d") + timedelta(days=+int(deltaDate))
                    
        f=L.Learner.remote(state, end_date.strftime("%Y-%m-%d"), predict_range,\
                           s0, e0, a0, i0, r0, d0, startNCases, ratio, weigthCases, weigthRecov, \
                           cleanRecovered, version, data, dead, savedata=False)
        result = f.train.remote() 
        result = ray.get(result) 

        del end_date,cleanRecovered, data, dead, point,f         

        gc.collect()

        return result
    return fobjective

In [9]:
@ray.remote(memory=50 * 1024 * 1024, max_calls=1)
def opt(state,s0,i0,e0,a0,r0,d0,wcases,date,startNCases, ratio, predict_range, version):

    bounds=[(0.8*s0,4*s0),(-1,1),(0.8*i0,4*i0),\
              (wcases*0.5,0.8),(.0001,.1)]
    maxiterations=500
    f=create_f(state,e0,a0,r0,d0,date,startNCases, ratio, predict_range, version)
    de = DE(f, bounds, maxiters=maxiterations)
    for step in de.geniterator():
        try:
            idx = step.best_idx
            norm_vector = step.population[idx]
            best_params = de.denormalize([norm_vector])
            del norm_vector, idx
        except:
            print("error in function evaluation")
    p=best_params[0]
    del f, bounds, data, dead
    
    return p

# Main Code

In [10]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines){
    return true;}

<IPython.core.display.Javascript object>

In [11]:
states=dfparam.state
display(states)
allStates=True
version="105"
gc.enable()

optimal=[]
if allStates:
    for state in states:
        #remove previous history file
        strFile='./results/history_'+state+version+'.csv'
        if os.path.isfile(strFile):
            os.remove(strFile)
        query = dfparam.query('state == "{}"'.format(state)).reset_index()
        parameters = np.array(query.iloc[:, 2:])[0]
        date,predict_range,s0,e0,a0,i0,r0,d0,startNCases,ratio,wcases,wrec = parameters
        optimal.append(opt.remote(state,s0,i0,e0,a0,r0,d0,wcases,date,startNCases, ratio, predict_range, version))        
else:
    state = "ES" 
    #remove previous history file
    strFile='./results/history_'+state+version+'.csv'
    if os.path.isfile(strFile):
        os.remove(strFile)
    query = dfparam.query('state == "{}"'.format(state)).reset_index()
    parameters = np.array(query.iloc[:, 2:])[0]
    start_date,predict_range,s0,e0,a0,i0,r0,d0,startNCases,ratio,wcases,wrec = parameters
    optimal.append(opt.remote(state,s0,i0,e0,a0,r0,d0,wcasesdate,startNCases, ratio, predict_range, version))        

0     SP
1     ES
2     MG
3     RJ
4     CE
5     PE
6     AM
7     PA
8     PI
9     RR
10    AP
Name: state, dtype: object

In [ ]:
optimal=ray.get(optimal)


2020-07-06 21:35:02,718	WARNING worker.py:1090 -- WARNING: 6 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.
2020-07-06 21:35:03,100	WARNING worker.py:1090 -- WARNING: 7 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.
2020-07-06 21:35:03,736	WARNING worker.py:1090 -- WARNING: 8 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.
2020-07-06 21:35:03,851	WARNING worker.py:1090 -- WARNING: 9 PYTHON workers have been started. This could be a resu

(pid=102699) /home/ats4i/anaconda3/envs/geo_env/lib/python3.6/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
(pid=102699)   warnings.warn(warning_msg, ODEintWarning)


2020-07-06 21:35:05,190	WARNING worker.py:1090 -- WARNING: 14 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.
2020-07-06 21:35:05,590	WARNING worker.py:1090 -- WARNING: 15 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.
2020-07-06 21:35:05,612	WARNING worker.py:1090 -- WARNING: 16 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.
2020-07-06 21:35:06,022	WARNING worker.py:1090 -- WARNING: 17 PYTHON workers have been started. This could be a 

(pid=102818) basinhopping step 0: f 6.6978e+08
(pid=102931) basinhopping step 0: f 8.22749e+07
(pid=102699) basinhopping step 0: f -2.58648e+14
(pid=102891) basinhopping step 0: f -4.80292e+11
(pid=102905) warning: basinhopping: local minimization failure
(pid=102905) basinhopping step 0: f 6.42086e+13
(pid=102931) basinhopping step 1: f 8.22749e+07 trial_f 3.46271e+11 accepted 0  lowest_f 8.22749e+07
(pid=102931) basinhopping step 2: f 8.14835e+07 trial_f 8.14835e+07 accepted 1  lowest_f 8.14835e+07
(pid=102931) found new global minimum on step 2 with function value 8.14835e+07
(pid=102891) basinhopping step 1: f -4.80292e+11 trial_f 3.70469e+08 accepted 0  lowest_f -4.80292e+11
(pid=102818) basinhopping step 1: f 6.6852e+08 trial_f 6.6852e+08 accepted 1  lowest_f 6.6852e+08
(pid=102818) found new global minimum on step 1 with function value 6.6852e+08
(pid=102700) basinhopping step 0: f -5.3499e+08
(pid=102762) basinhopping step 0: f -1.23696e+12
(pid=102891) warning: basinhopping: l

2020-07-06 21:36:01,660	WARNING worker.py:1090 -- WARNING: 22 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


...

(pid=126498) basinhopping step 3: f -3.44389e+12 trial_f 8.19496e+11 accepted 0  lowest_f -3.44389e+12
(pid=126498) basinhopping step 4: f -3.44389e+12 trial_f 8.19438e+11 accepted 0  lowest_f -3.44389e+12
(pid=126511) basinhopping step 0: f 5.76752e+09
(pid=126393) basinhopping step 8: f -3.4868e+12 trial_f 1.35742e+09 accepted 0  lowest_f -3.4868e+12
(pid=126418) basinhopping step 5: f -1.57862e+12 trial_f 9.1168e+08 accepted 0  lowest_f -1.57862e+12
(pid=126498) basinhopping step 5: f -3.44389e+12 trial_f 5.48647e+07 accepted 0  lowest_f -3.44389e+12
(pid=126498) /home/ats4i/anaconda3/envs/geo_env/lib/python3.6/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
(pid=126498)   warnings.warn(warning_msg, ODEintWarning)
(pid=126340) basinhopping step 8: f -7.10617e+12 trial_f 3.86816e+09 accepted 0  lowest_f -7.10617e+12
(pid=126366) /home/ats4i/anaconda3/envs/ge

2020-07-07 02:43:01,842	WARNING worker.py:1090 -- WARNING: 22 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


(pid=126393) basinhopping step 10: f -3.4868e+12 trial_f -1.72461e+12 accepted 0  lowest_f -3.4868e+12


2020-07-07 02:43:03,906	WARNING worker.py:1090 -- WARNING: 22 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.
2020-07-07 02:43:03,919	WARNING worker.py:1090 -- WARNING: 23 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


(pid=126472) basinhopping step 3: f -9.18461e+12 trial_f 1.84388e+10 accepted 0  lowest_f -9.18461e+12
(pid=126498) warning: basinhopping: local minimization failure
(pid=126498) basinhopping step 9: f -3.44389e+12 trial_f -2.0007e+12 accepted 0  lowest_f -3.44389e+12
(pid=126418) basinhopping step 8: f -1.57862e+12 trial_f 4.17546e+10 accepted 0  lowest_f -1.57862e+12
(pid=126471) basinhopping step 5: f -4.74422e+12 trial_f -4.71578e+12 accepted 0  lowest_f -4.74422e+12
(pid=126366) basinhopping step 10: f -1.59819e+13 trial_f 7.92692e+11 accepted 0  lowest_f -1.59819e+13


In [ ]:
for i in range(0,len(states)):    

    state=states[i]
    query = dfparam.query('state == "{}"'.format(state)).reset_index()
    parameters = np.array(query.iloc[:, 2:])[0]
    startdate,predict_range,s0,e0,a0,i0,r0,d0,startNCases,ratio,wcases,wrec = parameters
    Date = datetime.strptime(startdate, "%Y-%m-%d")
    end_date = Date + timedelta(days=+int(optimal[i][1]))
    dateStr=end_date.strftime("%Y-%m-%d")

    j = query['index'].values[0]
    dfparam.at[j, "s0"] = optimal[i][0]
    dfparam.at[j, "start-date"] = dateStr
    dfparam.at[j, "i0"] = optimal[i][2]
    dfparam.at[j, "WCASES"] = optimal[i][3]
    dfparam.at[j, "WREC"] = optimal[i][4]

    dfparam.to_csv("data/param_optimized_Yabox.csv", sep=",", index=False)
    dfparam
    